# Artificial Intelligence
## Assignment # 02
### Muhammad Zain Ali------22i-0562
### Syed Bilal Hassan Kazmi--22i-0505

## Implementation through UCS

In [1]:
from queue import PriorityQueue
import random

class State:
    def __init__(self, x, y, parent=None):
        self.x = x
        self.y = y
        self.parent = parent

    def __eq__(self, other):
        return self.x == other.x and self.y == other.y

class Node:
    def __init__(self, state, path_cost):
        self.state = state
        self.path_cost = path_cost

    def __lt__(self, other):
        return self.path_cost < other.path_cost

def is_valid(x, y, size):
    return 0 <= x < size and 0 <= y < size

def ucs(start, goal, forest_grid):
    size = len(forest_grid)
    visited = set()
    PQ = PriorityQueue()
    PQ.put(Node(start, 0))

    while not PQ.empty():
        current_node = PQ.get()
        current_state = current_node.state

        if current_state == goal:
            return current_node

        if (current_state.x, current_state.y) in visited:
            continue

        visited.add((current_state.x, current_state.y))

        x, y = current_state.x, current_state.y
        for dx, dy in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
            new_x, new_y = x + dx, y + dy

            if is_valid(new_x, new_y, size):
                new_state = State(new_x, new_y, current_node)

                # Check the type of grid encountered
                grid_type = forest_grid[new_x][new_y]
                if grid_type == 0:  # safegrid
                    new_cost = current_node.path_cost + random.randint(-3, -1)
                elif grid_type == -1:  # obstacle
                    new_cost = current_node.path_cost + random.randint(-4, -2)
                elif grid_type == 4:  # animal
                    if random.random() < 0.8:  # 80% chance of getting killed
                        new_cost = current_node.path_cost + random.randint(-4, -2)
                    else:
                        new_cost = current_node.path_cost + random.randint(-3, -1)
                elif grid_type == 5:  # impenetrable barrier
                    continue
                elif new_x == size - 1:
                    new_cost = current_node.path_cost - 1
                    new_x, new_y = x - dx, y - dy
                else:
                    new_cost = current_node.path_cost + random.randint(-3, -1)

                PQ.put(Node(new_state, new_cost))

    return None

def print_path_and_actions(path):
    print("Path:")
    for i, (x, y) in enumerate(reversed(path)):  # Iterate in reverse to print from start to end
        print(f"({x}, {y})", end=" ")
        if (i + 1) % 5 == 0: 
            print()
    actions = [(abs(path[i][0] - path[i-1][0]), abs(path[i][1] - path[i-1][1])) for i in range(1, len(path))]
    total_cost = sum(sum(action) for action in actions)
    print("\n__________________________________")
    print("Total cost of the path:", total_cost)
    print("__________________________________")
    print("Grid representation:")

def print_grid_with_path(forest_grid, path):
    size = len(forest_grid)
    for i in range(size):
        for j in range(size):
            if (i, j) in path:
                print(" *", end=" ")
            else:
                print(f"{forest_grid[i][j]:2}", end=" ")
        print()

class ForestGrid:
    def __init__(self, size=8):
        self.size = size
        self.grid = self.generate_grid(size)

    def generate_grid(self, size):
        grid = []
        for i in range(size):
            row = []
            for j in range(size):
                cell_type = random.choice([0, 0, 0, 0, 0, -1, 5, 4])
                row.append(cell_type)
            grid.append(row)
        start_x, start_y = 0, 0  # northwest corner
        goal_x, goal_y = size - 1, size - 1  # southeast corner
        grid[start_x][start_y] = 2  # Start
        grid[goal_x][goal_y] = 3  # Goal
        return grid

    def print_grid(self):
        print("__________________________________")
        for row in self.grid:
            for cell in row:
                print(f"{cell:2}", end=" ")
            print()
        print("__________________________________")

def main():
    size = 8
    forest = ForestGrid(size)
    print("Generated Forest Grid:")
    forest.print_grid()

    start_state = State(0, 0) # northwest corner
    goal_state = State(size - 1, size - 1) # southeast corner

    result = ucs(start_state, goal_state, forest.grid)

    if result is not None:
        path = []
        node = result 
        while node is not None:
            path.append((node.state.x, node.state.y))
            node = node.state.parent
        
        print_path_and_actions(path)
        print_grid_with_path(forest.grid, path)
    else:
        print("Goal is unreachable.")

if __name__ == "__main__":
    main()


Generated Forest Grid:
__________________________________
 2  0  4 -1  4  4  0  0 
 0 -1 -1  0  0  4 -1  4 
 0 -1 -1 -1  4  0  0  0 
 0  0 -1  0 -1  0 -1  0 
 0  0 -1  5  0  5  5  0 
-1  0  0  0  0 -1  0 -1 
 0  0 -1  0  5  5  0 -1 
 5  0  0  0  0  0  5  3 
__________________________________
Path:
(0, 0) (1, 0) (1, 1) (1, 2) (2, 2) 
(3, 2) (4, 2) (5, 2) (6, 2) (6, 1) 
(7, 1) (7, 2) (7, 3) (6, 3) (5, 3) 
(5, 4) (5, 5) (5, 6) (5, 7) (6, 7) 
(7, 7) 
__________________________________
Total cost of the path: 20
__________________________________
Grid representation:
 *  0  4 -1  4  4  0  0 
 *  *  *  0  0  4 -1  4 
 0 -1  * -1  4  0  0  0 
 0  0  *  0 -1  0 -1  0 
 0  0  *  5  0  5  5  0 
-1  0  *  *  *  *  *  * 
 0  *  *  *  5  5  0  * 
 5  *  *  *  0  0  5  * 


## Implementation through A* Search

In [2]:
from queue import PriorityQueue
import random

class State:
    def __init__(self, x, y, parent=None):
        self.x = x
        self.y = y
        self.parent = parent

    def __eq__(self, other):
        return self.x == other.x and self.y == other.y

class Node:
    def __init__(self, state, path_cost, heuristic_cost):
        self.state = state
        self.path_cost = path_cost
        self.heuristic_cost = heuristic_cost

    def __lt__(self, other):
        return (self.path_cost + self.heuristic_cost) < (other.path_cost + other.heuristic_cost)

def is_valid(x, y, size):
    return 0 <= x < size and 0 <= y < size

def manhattan_distance(state, goal):
    return abs(state.x - goal.x) + abs(state.y - goal.y)

def a_star(start, goal, forest_grid):
    size = len(forest_grid)
    visited = set()
    PQ = PriorityQueue()
    PQ.put(Node(start, 0, manhattan_distance(start, goal)))

    while not PQ.empty():
        current_node = PQ.get()
        current_state = current_node.state

        if current_state == goal:
            return current_node

        if (current_state.x, current_state.y) in visited:
            continue

        visited.add((current_state.x, current_state.y))

        x, y = current_state.x, current_state.y
        for dx, dy in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
            new_x, new_y = x + dx, y + dy

            if is_valid(new_x, new_y, size):
                new_state = State(new_x, new_y, current_node)

                # Check the type of grid encountered
                grid_type = forest_grid[new_x][new_y]
                if grid_type == 0:  # safegrid
                    new_cost = current_node.path_cost + random.randint(-3, -1)
                elif grid_type == -1:  # obstacle
                    new_cost = current_node.path_cost + random.randint(-4, -2)
                elif grid_type == 4:  # animal
                    if random.random() < 0.8:  # 80% chance of getting killed
                        new_cost = current_node.path_cost + random.randint(-4, -2)
                    else:
                        new_cost = current_node.path_cost + random.randint(-3, -1)
                elif grid_type == 5:  # impenetrable barrier
                    continue
                elif new_x == size - 1:
                    new_cost = current_node.path_cost - 1
                    new_x, new_y = x - dx, y - dy
                else:
                    new_cost = current_node.path_cost + random.randint(-3, -1)

                PQ.put(Node(new_state, new_cost, manhattan_distance(new_state, goal)))

    return None

def print_path_and_actions(path):
    print("Path:")
    for i, (x, y) in enumerate(reversed(path)):
        print(f"({x}, {y})", end=" ")
        if (i + 1) % 5 == 0:
            print()
    actions = [(abs(path[i][0] - path[i-1][0]), abs(path[i][1] - path[i-1][1])) for i in range(1, len(path))]
    total_cost = sum(sum(action) for action in actions)
    print("\n__________________________________")
    print("Total cost of the path:", total_cost)
    print("__________________________________")
    print("Grid representation:")

def print_grid_with_path(forest_grid, path):
    size = len(forest_grid)
    for i in range(size):
        for j in range(size):
            if (i, j) in path:
                print(" *", end=" ")
            else:
                print(f"{forest_grid[i][j]:2}", end=" ")
        print()

class ForestGrid:
    def __init__(self, size=8):
        self.size = size
        self.grid = self.generate_grid(size)

    def generate_grid(self, size):
        grid = []
        for i in range(size):
            row = []
            for j in range(size):
                cell_type = random.choice([0, 0, 0, 0, 0, -1, 5, 4])
                row.append(cell_type)
            grid.append(row)
        start_x, start_y = 0, 0  # northwest corner
        goal_x, goal_y = size - 1, size - 1  # southeast corner
        grid[start_x][start_y] = 2  # Start
        grid[goal_x][goal_y] = 3  # Goal
        return grid

    def print_grid(self):
        print("__________________________________")
        for row in self.grid:
            for cell in row:
                print(f"{cell:2}", end=" ")
            print()
        print("__________________________________")

def main():
    size = 8
    forest = ForestGrid(size)
    print("Generated Forest Grid:")
    forest.print_grid()

    start_state = State(0, 0) # northwest corner
    goal_state = State(size - 1, size - 1) # southeast corner

    result = a_star(start_state, goal_state, forest.grid)

    if result is not None:
        path = []
        node = result
        while node is not None:
            path.append((node.state.x, node.state.y))
            node = node.state.parent
        
        print_path_and_actions(path)
        print_grid_with_path(forest.grid, path)
    else:
        print("Goal is unreachable.")

if __name__ == "__main__":
    main()


Generated Forest Grid:
__________________________________
 2 -1  0  5  0  0  4  0 
 0  0  4  4  0  0  4  0 
 0  4  0  5  0  0  0  0 
-1  0  0  0  0  4  0  0 
 0  0  0  5  0  0 -1  5 
 0  5 -1  0  5  0  0 -1 
 0  0  0  0  0  0  0  4 
 0  0  0  5  4  4  0  3 
__________________________________
Path:
(0, 0) (0, 1) (1, 1) (1, 2) (1, 3) 
(1, 4) (2, 4) (3, 4) (3, 5) (3, 6) 
(4, 6) (5, 6) (5, 7) (6, 7) (7, 7) 

__________________________________
Total cost of the path: 14
__________________________________
Grid representation:
 *  *  0  5  0  0  4  0 
 0  *  *  *  *  0  4  0 
 0  4  0  5  *  0  0  0 
-1  0  0  0  *  *  *  0 
 0  0  0  5  0  0  *  5 
 0  5 -1  0  5  0  *  * 
 0  0  0  0  0  0  0  * 
 0  0  0  5  4  4  0  * 


### Test Cases:
1.
[
 [2, 0, 0, 0, 0],
 [4, 0, 0, -1, 0],
 [0, -1, 0, 0, 0],
 [0, 0, -1, 0, 0],
 [0, 0, 0, -1, 3]
]
2.
[
 [2, 0, 0, 0, 0],
 [0, 0, 0, -1, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 5]
]
3.
[
 [2, 0, 0, 0, 0],
 [0, -1, 0, 0, 0],
 [0, 0, -1, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, -1, 3]
]
